In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import json

/home/tu/tu_tu/tu_bbpgo01/miniconda3/envs/l/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
NER = ["EFFECT"]
set = "test"

In [3]:
l = []
for ner in NER:
	test_sentences = pd.read_json(f"../NER/{ner}/test.json", lines=True)
	test_output = pd.read_json(f"../NER_output/{ner}/test_outputs.json")
	merged = pd.merge(test_sentences, test_output, left_index=True, right_index=True)
	merged.loc[:,"ner"] = ner
	l.append(merged)

df = pd.concat(l)

In [4]:
df = df.reset_index().drop(columns=["index"])

In [5]:
df

,tokens,ner_tags,id,predictions,label_ids,word_ids,ner
0,"[Taking, into, consideration, the, above, evid...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[[-1.8748829364776611, -1.967904806137085, 4.0...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",EFFECT
1,"[The, two, draft, genomes, (, Nitrospira, sp.,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1,"[[-2.734492301940918, -2.349016666412353, 5.28...","[-100, 2, 2, 2, 2, 2, 2, -100, -100, 2, -100, ...","[None, 0, 1, 2, 3, 4, 5, 5, 5, 6, 6, 7, 8, 8, ...",EFFECT
2,"[Sle1, is, a, substrate, of, the, ClpXP, prote...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2,"[[-1.911755681037902, -1.535628080368042, 3.48...","[-100, 2, -100, 2, 2, 2, 2, 2, 2, -100, -100, ...","[None, 0, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 9, ...",EFFECT
3,"[In, this, study, ,, we, sequenced, and, assem...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",3,"[[-2.416978597640991, -2.321799755096435, 5.10...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100, -10...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 9, 10,...",EFFECT
4,"[MF59, is, an, oil-in-water, emulsion, with, a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",4,"[[-2.241502523422241, -2.455163955688476, 4.54...","[-100, 2, -100, 2, 2, 2, -100, -100, -100, -10...","[None, 0, 0, 1, 2, 3, 3, 3, 3, 3, 4, 5, 6, 7, ...",EFFECT
...,...,...,...,...,...,...,...
411,"[For, the, housekeeping, gene, GAPDH, ,, 20, s...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",411,"[[-2.416497945785522, -1.909170269966125, 4.49...","[-100, 2, 2, 2, -100, -100, 2, 2, 2, 2, 2, -10...","[None, 0, 1, 2, 2, 2, 3, 4, 5, 6, 7, 7, 7, 8, ...",EFFECT
412,"[Chemotaxonomic, analyses, showed, that, type,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",412,"[[-2.4317431449890132, -2.126706838607788, 4.5...","[-100, 2, -100, -100, 2, 2, 2, 2, 2, 2, -100, ...","[None, 0, 0, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, ...",EFFECT
413,"[[, 6, ], Some, volatile, sesquiterpenoids, ar...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",413,"[[-2.476452589035034, -1.915351033210754, 3.87...","[-100, 2, 2, 2, 2, 2, 2, -100, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 5, 6, 7, 8, 9, 10, 10...",EFFECT
414,"[Nanosynbacter, lyticus, stain, TM7x, of, the,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",414,"[[-2.470869064331054, -2.453198671340942, 5.44...","[-100, 2, -100, -100, -100, -100, 2, -100, 2, ...","[None, 0, 0, 0, 0, 0, 1, 1, 2, 3, 3, 3, 4, 5, ...",EFFECT


In [6]:
label_ids = df[["label_ids"]]

In [7]:
annotations = []
for j in range(len(df)):
	for i in zip(df.loc[j,"tokens"],df.loc[j,"ner_tags"]):
		if i[1] != "O":
			annotations.append({"label":i[1],"text":i[0]})

### Predict sentences again

In [8]:
ner = "EFFECT"

In [9]:
path= f"../NER_output/{ner}"

tokenizer = AutoTokenizer.from_pretrained(path, model_max_length=512)
model = AutoModelForTokenClassification.from_pretrained(path)

nlp = pipeline(task='ner', model=model, tokenizer=tokenizer,
               aggregation_strategy="average", device=1)


In [10]:
import jsonlines

# Specify the path to the JSONL file
jsonl_file = f"../NER/{ner}/test.jsonls"

predictions = []
# Open the JSONL file
with jsonlines.open(jsonl_file) as reader:
    # Iterate over each line in the file
    for line in reader:
        for l in line:
            # Get the value from the "text" field
            text = l["data"]["text"]
            # Pass the value to the nlp() function
            print(text)
            predict = nlp(text)
            predictions.append(predict)
            print(predict)

Taking into consideration the above evidence and uncertainties, the Panel considers that the possibility of entry into the nursery infecting apple plants from surrounding orchards may be unlikely.
[]
The two draft genomes (Nitrospira sp. strains UW-LDO-01 and UW-LDO-02) had 3.9 and 3.5 Mbp in total with average GC contents of 54.9% and 59.2%, respectively (see Table S1 in the supplemental material).
[]
Sle1 is a substrate of the ClpXP protease, and while the high Sle1 levels in cells lacking ClpXP activity confer β-lactam hyper-resistance, USA300 cells lacking Sle1 are as susceptible to β-lactams as cells lacking mecA.
[]
In this study, we sequenced and assembled the whole-genome of P . agglomerans KM1 isolated from kimchi in South Korea.
[]
MF59 is an oil-in-water emulsion with a squalene internal oil phase and an external aqueous phase.[
[]
Pop5, a phazolicin producer, and another of Rhizobium anhuiense T24, a trifolitoxin producer.
[]
Indeed, also [64] reported that the salt toleran

/home/tu/tu_tu/tu_bbpgo01/miniconda3/envs/l/lib/python3.12/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[]
However, when noise was added to the distance and duration cues as in Experiment 2, listeners were able to make use of speed information though performance was worse.
[]
The number of biofilm cells for strains, ATCC 19606, R3, R4, R6, R8, and R9, decreased in the presence of TGC in a dose-dependent manner (Fig 1D and 1L) (S2H, S2P, S3H and S3L Figs), as well as those in strains R2, R1, R5, R7, and R10 were significantly decreased in the presence of TGC at 1/2 MIC (Fig 1H) (S2D, S2L, S3D and S3P Figs).
[]
Coabalamin-dependent O-demethylase in Blautia sp. strain MRG-PMF1 was found to catalyze the unprecedented allyl aryl ether cleavage reaction.
[]
ZH07 was grown at 30°C on inorganic phosphorus medium or egg yolk medium for 7 days.
[]
Here, we report the high-quality draft genome sequence of JSC-12, which may help elucidate the mechanisms of resistance to extreme iron concentrations in siderophilic cyanobacteria and lead to new remediation biotechnologies.
[]
ISR activity in arginyl-t

In [11]:
predictions = [p for p in predictions if p]

In [12]:
predictions

[[{'entity_group': 'I',
   'score': 0.4631672,
   'word': 'tolerance',
   'start': 41,
   'end': 50}],
 [{'entity_group': 'I',
   'score': 0.55776197,
   'word': 'promoting neuronal health',
   'start': 203,
   'end': 228}],
 [{'entity_group': 'I',
   'score': 0.5178823,
   'word': 'lateral root formation',
   'start': 33,
   'end': 55}],
 [{'entity_group': 'I',
   'score': 0.513415,
   'word': 'primary ciliogenesis',
   'start': 64,
   'end': 84},
  {'entity_group': 'I',
   'score': 0.45505422,
   'word': 'ciliogenesis',
   'start': 107,
   'end': 119},
  {'entity_group': 'I',
   'score': 0.51803035,
   'word': 'ciliary growth',
   'start': 165,
   'end': 179},
  {'entity_group': 'I',
   'score': 0.4617919,
   'word': 'ciliogenesis',
   'start': 247,
   'end': 259}],
 [{'entity_group': 'I',
   'score': 0.45611012,
   'word': '- acid import',
   'start': 53,
   'end': 65}],
 [{'entity_group': 'I',
   'score': 0.54359263,
   'word': 'er stress',
   'start': 132,
   'end': 141},
  {'enti

In [13]:
annotations

[{'label': 'B', 'text': 'activity'},
 {'label': 'B', 'text': 'β-lactam'},
 {'label': 'I', 'text': 'hyper-resistance'},
 {'label': 'B', 'text': 'resistance'},
 {'label': 'I', 'text': 'to'},
 {'label': 'I', 'text': 'extreme'},
 {'label': 'I', 'text': 'iron'},
 {'label': 'I', 'text': 'concentrations'},
 {'label': 'B', 'text': 'ISR'},
 {'label': 'B', 'text': 'neuroprotective'},
 {'label': 'I', 'text': 'effects'},
 {'label': 'B', 'text': 'neuronal'},
 {'label': 'I', 'text': 'health'},
 {'label': 'B', 'text': 'lateral'},
 {'label': 'I', 'text': 'root'},
 {'label': 'I', 'text': 'formation'},
 {'label': 'B', 'text': 'auxin'},
 {'label': 'I', 'text': 'biosynthesis'},
 {'label': 'B', 'text': 'confers'},
 {'label': 'I', 'text': 'resistance'},
 {'label': 'I', 'text': 'to'},
 {'label': 'I', 'text': 'Pb'},
 {'label': 'I', 'text': '('},
 {'label': 'I', 'text': 'II'},
 {'label': 'I', 'text': ')'},
 {'label': 'I', 'text': 'salts'},
 {'label': 'B', 'text': 'non-adherent'},
 {'label': 'I', 'text': 'growi

In [14]:
for prediction in predictions:
    pred = prediction[0]["word"]
    count = 0
    for annotation in annotations:
        ann = annotation["text"]
        if ann in pred:
            count +=1
    if count <1:
        print(pred)

In [15]:
ners =["COMPOUND", "DISEASE", "EFFECT","ISOLATE","MEDIUM","METABOLITE","ORGANISM",
            "PHENOTYPE","PROTEIN","SPECIES","STRAIN"]
for ner in ners:
    test_sentences = pd.read_json(f"../NER/{ner}/test.json", lines=True)
    test_output = pd.read_json(f"../NER_output/{ner}/test_outputs.json")
    merged = pd.merge(test_sentences, test_output, left_index=True, right_index=True)
    merged.loc[:,"ner"] = ner
    label_ids = merged[["label_ids"]]

    annotations = []
    for j in range(len(merged)):
    	for i in zip(merged.loc[j,"tokens"],merged.loc[j,"ner_tags"]):
    		if i[1] != "O":
    			annotations.append({"label":i[1],"text":i[0]})
    
    path= f"../NER_output/{ner}"
    
    tokenizer = AutoTokenizer.from_pretrained(path, model_max_length=512)
    model = AutoModelForTokenClassification.from_pretrained(path)
    
    nlp = pipeline(task='ner', model=model, tokenizer=tokenizer,
                   aggregation_strategy="average", device=1)

    # Specify the path to the JSONL file
    jsonl_file = f"../NER/{ner}/test.jsonls"
    
    predictions = []
    # Open the JSONL file
    with jsonlines.open(jsonl_file) as reader:
        # Iterate over each line in the file
        for line in reader:
            for l in line:
    
                # Get the value from the "text" field
                text = l["data"]["text"]
                # Pass the value to the nlp() function
                predict = nlp(text)
                predictions.append(predict)
    predictions = [p for p in predictions if p]
    print("-----")
    print(ner)
    print("-----")
    for prediction in predictions:
        pred = prediction[0]["word"]
        count = 0
        for annotation in annotations:
            ann = annotation["text"].lower()
            if ann in pred:
                count +=1
        if count ==0:
            print(pred)

/home/tu/tu_tu/tu_bbpgo01/miniconda3/envs/l/lib/python3.12/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


-----
COMPOUND
-----
doripenem
h
el42
h
ant44
n
co
h
nemafric
o
co
5


/home/tu/tu_tu/tu_bbpgo01/miniconda3/envs/l/lib/python3.12/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


-----
DISEASE
-----
uc
lung
axial


/home/tu/tu_tu/tu_bbpgo01/miniconda3/envs/l/lib/python3.12/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

-----
PROTEIN
-----
mrp
β
α
e
np
α


/home/tu/tu_tu/tu_bbpgo01/miniconda3/envs/l/lib/python3.12/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


-----
SPECIES
-----
a
p
p
m
l
m
r
a
e
a
n
m
p
s
p
s
r
l
k
v
e
l
s
a
s
x
p
metallidurans
x
e
m
a
flexner
m
v


/home/tu/tu_tu/tu_bbpgo01/miniconda3/envs/l/lib/python3.12/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


-----
STRAIN
-----
s
l
p
b
l
v
osish
l
rh
vu
p
osish
b
b
gt
m
n
t
m
p
s
p
y
b
v
b
k
shli
l
l
b
b
l
p
db
b
s
v
osish
v
p
b
b
p
g
b
v
d
osish
p
w
b
v
br
osish
osish
l
p
p
